In [ ]:
import pandas as pd
import plotly.graph_objects as go
from datetime import date, timedelta, datetime
import requests
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

In [ ]:
date_time_str = datetime.today().strftime('%Y-%m-%d')
today = datetime.strptime(f'{date_time_str}', '%Y-%m-%d')

start = today - timedelta(days = 6)
end = today

print(f'start:{start:} and end:{end}')

In [ ]:
# latest = requests.get('https://pselookup.vrymel.com/api/stocks').json()["stocks"]
# ticker_codes = [x['ticker_symbol'] for x in latest]
# print(ticker_codes)

# with open("ticker_codes.csv", "w") as f:
#     f.write(','.join(ticker_codes))

ticker_codes = []
with open("ticker_codes.csv", "r") as f:
    ticker_codes = f.readline().split(',') 
#     
print(ticker_codes)

In [ ]:
#reference https://www.stockswechart.com/pse-chart/blue-chip-stocks-pse/
# ticker_codes = [ "MFC", "SMPH", "SM", "SLF", "ALI", "FB", "AC", "BDO", "JGS", "MER", "SMC", "BPI",
#                 "JFC", "URC", "AEV", "MBT", "ICT", "AP", "GLO", "TEL", "HVN", "MEG", "GTCAP", "LTG", 'CEB'
#                 "AGI", "DMC", "MPI", "PGOLD", "BLOOM", "FDC", "MAC", 'LOTO', 'FLI', 'MM', 'DITO', 'PHR', 'ACEN' ]
ticker_codes = ['PSB', 'CNPF', 'PBB', 'ALHI', 'IMI', 'KPPI', 'KEP', 'SHNG', 'I', 'ROX', 'IMP', 'BRN', 'LSC', 'T', 'SSP', 'BKR', 'ION', 'LC', 'APC', 'SFI', 'LIHC', 'BSC', 'LAND', 'IS', 'MB', 'FOOD', 'APO', 'PLC', 'TFC', 'CHIB', 'ABG', 'AB', 'ALLHC', 'FJP', 'PA', 'PHA', 'TUGS', 'WIN', 'ABA', 'MED', 'MRC', 'RLT', 'DD', 'LOTO', 'CEI', 'NI', 'REG', 'PRC', 'BEL', 'CEB', 'WPI', 'PGOLD', 'MRSGI', 'OPM', 'PIZZA']
print(ticker_codes)

In [ ]:

bullish_eng_codes = []
bearish_eng_codes = []
bullish_swing_codes = []
bearish_swing_codes = []

bullish_pinbar = []
bearish_pinbar = []
outside_bar = []
inside_bar = []
bull_hammer = []
bear_hammer = []
doji = []
    
my_request = f'https://pselookup.vrymel.com/api/stocks/{code}/history/{start}/{end}'
print(f'start:{start} and end:{end}, request {my_request}')
for code in ticker_codes:
    # Get updated value from phisix-api
    try:
        print(f'looking up for {code}...')
        latest = requests.get(f'http://phisix-api.appspot.com/stocks/{code}.json', timeout=5).json()
        last_price = latest['stock'][0]['price']['amount']
        date = latest['as_of']

        #get previos data from pselookup
        data = requests.get(f'https://pselookup.vrymel.com/api/stocks/{code}/history/{start}/{end}', timeout=5).json()['history']
        data[-1]['close']=last_price

        df = pd.json_normalize(data)
        oo_indexed = df.set_index('trading_date')


        if oo_indexed.shape[0] < 3:
            print('cannot analyze data, adjust start date')
            continue
        
        for i in range(2,oo_indexed.shape[0]):
            current = oo_indexed.iloc[i,:]
            prev = oo_indexed.iloc[i-1,:]
            prev_2 = oo_indexed.iloc[i-2,:]

            if current['open'] < current['close']:
                candle_type='Bull'
                head_wick = current['high'] - current['close']
                tail_wick = current['open'] - current['low']
            elif current['open'] == current['close']: 
                doji.append(f'{code}:{current.name}')
            else: 
                candle_type='Bear'
                head_wick = current['high'] - current['open']
                tail_wick = current['close'] - current['low']

            realbody = abs(current['open'] - current['close'])
            candle_range = current['high'] - current['low']

            # Bullish Engulfing
            if current['high'] > prev['high'] and current['low'] < prev['low'] and realbody >= 0.8 * candle_range and current['close'] > current['open']:
                bullish_eng_codes.append(f'{code}:{current.name}')

            # Bearish Engulfing
            if current['high'] > prev['high'] and current['low'] < prev['low'] and realbody >= 0.8 * candle_range and current['close'] < current['open']:
                bearish_eng_codes.append(f'{code}:{current.name}')

            # Bullish Swing
            if current['low'] > prev['low'] and prev['low'] < prev_2['low']:
                bullish_swing_codes.append(f'{code}:{current.name}')

            # Bearish Swing
            if current['high'] < prev['high'] and prev['high'] > prev_2['high']:
                bearish_swing_codes.append(f'{code}:{current.name}')

            # Bullish Pinbar
            if realbody <= candle_range/3 and  min(current['open'], current['close']) > (current['high'] + current['low'])/2 and current['low'] < prev['low']:
                bullish_pinbar.append(f'{code}:{current.name}')

            # Bearish Pinbar
            if realbody <= candle_range/3 and max(current['open'] , current['close']) < (current['high'] + current['low'])/2 and current['high'] > prev['high']:
                bearish_pinbar.append(f'{code}:{current.name}')

            # Inside bar
            if current['high'] < prev['high'] and current['low'] > prev['low']:
                inside_bar.append(f'{code}:{current.name}')

            # Outside bar
            if current['high'] > prev['high'] and current['low'] < prev['low']:
                outside_bar.append(f'{code}:{current.name}')

            # Hammer
            if realbody > head_wick*2 and tail_wick > head_wick*2 and realbody*2 < tail_wick:
                if candle_type == 'Bull':
                    bull_hammer.append(f'{code}:{current.name}')
                else:
                    bear_hammer.append(f'{code}:{current.name}')
    except Exception as e:
        print(f'Failed for Code{code}, {data}, {e}')
        continue
            
                
print('=========== BULLS =============')
print('Bullish Swing')
print(bullish_swing_codes)

print('Bullish Pinbar')
print(bullish_pinbar)

print('Bullish Engulfing')
print(bullish_eng_codes)

print('Bullish Hammer')
print(bull_hammer)

print('========== BEARS ==============')

print('Bearish Swing')
print(bearish_swing_codes)

print('Bearish Pinbar')
print(bearish_pinbar)

print('Bearish Engulfing')
print(bearish_eng_codes)

print('Bearish Hammer')
print(bear_hammer)

print('==============================')

print('Outside bar')
print(outside_bar)

print('Inside bar')
print(inside_bar)

In [ ]:
                
print('=========== BULLS =============')
print('Bullish Swing')
print(bullish_swing_codes)

print('Bullish Pinbar')
print(bullish_pinbar)

print('Bullish Engulfing')
print(bullish_eng_codes)

print('Bullish Hammer')
print(bull_hammer)

print('========== BEARS ==============')

print('Bearish Swing')
print(bearish_swing_codes)

print('Bearish Pinbar')
print(bearish_pinbar)

print('Bearish Engulfing')
print(bearish_eng_codes)

print('Bearish Hammer')
print(bear_hammer)

print('==============================')

print('Outside bar')
print(outside_bar)

print('Inside bar')
print(inside_bar)

In [ ]:
# df = pd.json_normalize(data)
# layout = dict(
#             title=f"{STOCK_CODE} Stock Data",
#             xaxis = dict(
#             type="category", 
#             categoryorder='category ascending'))
# fig = go.Figure(data=[go.Candlestick(x=df['trading_date'],
#                 open=df['open'],
#                 high=df['high'],
#                 low=df['low'],
#                 close=df['close'])], layout=layout)
# fig.show()

In [ ]:
df.head()

In [ ]:
oo_indexed = df.set_index('trading_date')
# oo_indexed.volume.plot(kind='bar',figsize=(20,10))

In [ ]:
oo_indexed.head(3)

In [ ]:
for i in range(2,oo_indexed.shape[0]):
    current = oo_indexed.iloc[i,:]
    prev = oo_indexed.iloc[i-1,:]
    prev_2 = oo_indexed.iloc[i-2,:]

    realbody = abs(current['open'] - current['close'])
    candle_range = current['high'] - current['low']


    if current['open'] < current['close']:
        candle_type='Bull'
        head_wick = current['high'] - current['close']
        tail_wick = current['open'] - current['low']
    elif current['open'] == current['close']: 
        oo_indexed.loc[idx, 'Doji'] = True
    else: 
        candle_type='Bear'
        head_wick = current['high'] - current['open']
        tail_wick = current['close'] - current['low']

    idx = oo_indexed.index[i]

    # Bullish swing
#         oo_indexed.loc[idx,'Bullish swing'] = current['low'] > prev['low'] and prev['low'] < prev_2['low']

#         # Bearish swing
#         oo_indexed.loc[idx,'Bearish swing'] = current['high'] < prev['high'] and prev['high'] > prev_2['high']

#         # Bullish pinbar
#         oo_indexed.loc[idx,'Bullish pinbar'] = realbody <= candle_range/3 and  min(current['open'], current['close']) > (current['high'] + current['low'])/2 and current['low'] < prev['low']

#         # Bearish pinbar
#         oo_indexed.loc[idx,'Bearish pinbar'] = realbody <= candle_range/3 and max(current['open'] , current['close']) < (current['high'] + current['low'])/2 and current['high'] > prev['high']

#         # Inside bar
#         oo_indexed.loc[idx,'Inside bar'] = current['high'] < prev['high'] and current['low'] > prev['low']

#         # Outside bar
#         oo_indexed.loc[idx,'Outside bar'] = current['high'] > prev['high'] and current['low'] < prev['low']

    # Bullish engulfing
    oo_indexed.loc[idx,'Bullish engulfing'] = current['high'] > prev['high'] and current['low'] < prev['low'] and realbody >= 0.8 * candle_range and current['close'] > current['open']

    # Bearish engulfing
    oo_indexed.loc[idx,'Bearish engulfing'] = current['high'] > prev['high'] and current['low'] < prev['low'] and realbody >= 0.8 * candle_range and current['close'] < current['open']


    # Bearish Hammer
    if realbody > head_wick*2 and tail_wick > head_wick*2:
        oo_indexed.loc[idx,f'{candle_type}ish hammer'] = realbody*2 < tail_wick


#         oo_indexed.fillna(False, inplace=True)


In [ ]:
bull_engulfing = oo_indexed[oo_indexed['Bullish engulfing'] == True]
bull_engulfing

In [ ]:
bear_engulfing = oo_indexed[oo_indexed['Bearish engulfing'] == True]
bear_engulfing

In [ ]:
bull_hammer = oo_indexed[oo_indexed['Bullish hammer'] == True]
bull_hammer

In [ ]:
bear_hammer = oo_indexed[oo_indexed['Bearish hammer'] == True]
bear_hammer